In [4]:
import os
import subprocess
import json
import numpy as np
import sklearn.metrics
import sys
from run_rrc_python import run_rrc_python

In [5]:
def evaluate(tasks, berts, domains, runs=10, dropout=0.0, epsilon=0.0):
    for task in tasks:
        for bert in berts:
            for domain in domains:
                scores=[]
                run_rrc_python(task, bert, domain, "pt_rrc", runs, "0", dropout, epsilon)
                for run in range(1, runs+1):
                    DATA_DIR=os.path.join(task, domain)
                    OUTPUT_DIR=os.path.join("run", bert+"_"+task, domain, str(run) )
                    if "rrc" in task:
                        ret=!python ../eval/evaluate-v1.1.py ../$DATA_DIR/test.json ../$OUTPUT_DIR/predictions.json
                        score=json.loads(ret[0])
                        scores.append([score["exact_match"], score["f1"] ] )
                    else:
                        raise Exception("unknown task")
                scores=np.array(scores)
                m=scores.mean(axis=0)

                if "rrc" in task:
                    em, f1 = m
                    return em, f1

In [8]:
epochs = list(range(3, 11))
dropouts = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
epsilons = [0.0, 0.2, 0.5, 1.0, 2.0, 5.0]

tasks=["rrc"]
berts=["pt"]
domains=["laptop"]

print("epoch dropout epsilon em f1")
for dropout in dropouts:
    for epsilon in epsilons:
        em, f1 = evaluate(tasks, berts, domains, 3, dropout, epsilon)
        print("3 {} {} {} {}".format(dropout, epsilon, round(em, 2), round(f1, 2)))


epoch dropout epsilon em f1
3 0.0 0.0 51.38 68.13
